# Thruster angles

# Purpose
Check that thruster angles are post processed correctly, from the cos/sin

# Methodology
* Load data from one trip and see that the thruster angles makes sense.

# Setup

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips
import scipy.integrate
import seaborn as sns


## Load one trip:

In [ ]:
#df = prepare_dataset.get_dataset(do_calculate_rudder_angles=False, n_rows=900)
trip = get_dataset.get_trip(trip_no=3)

In [ ]:
trip.head()

In [ ]:
visualize.plot_map(df=trip, width=1000, height=600, zoom_start=14)

In [ ]:
trip2 = trip.resample('5S').mean()
trip2 = prepare_dataset.calculate_rudder_angles(df=trip2, drop=False)
#trip2 = trip2.iloc[0:500]

In [ ]:
deltas = ['delta_%i' % i for i in range(1,5)]

fig,axes=plt.subplots(ncols=len(deltas))

for delta,ax in zip(deltas,axes):

    trip2[delta].plot(ax=ax)
    ax.legend(loc='upper left')    
    ax.grid(True)


In [ ]:
thruster_powers = ['power_em_thruster_%i' % i for i in range(1,5)]

fig,axes=plt.subplots(ncols=len(thruster_powers))

for thruster_power,ax in zip(thruster_powers,axes):

    trip[thruster_power].plot(ax=ax)
    ax.legend(loc='upper left')

In [ ]:
from ipywidgets import interactive

In [ ]:
trip_ = trip.copy()
trip_ = prepare_dataset.calculate_rudder_angles(df=trip_, drop=False)
trip_['trip_time_s'] = pd.TimedeltaIndex(trip_['trip_time']).total_seconds()
#trip_.set_index('trip_time_s', inplace=True)

In [ ]:
def plot_thruster(ax, x,y,cos,sin,l=10):
    
    ax.arrow(x=x, y=y, dx=-l*sin, dy=-l*cos, head_width=l/5, head_length=l/5)
    
def plot_thrust_allocation(row, lpp=50, beam=20):
    
    fig,axes=plt.subplots(ncols=2)
    fig.set_size_inches(20,5)
    ax = axes[0]    
    positions = np.array([
        [-1,-1],
        [1,-1],
        [1,1],
        [-1,1],
        
    ])
    
    coordinates = np.array([positions[:,0]*beam/2, positions[:,1]*lpp/2] ).T
        
    
    for i,position in enumerate(coordinates):
        
        n=i+1
        sin_key = 'sin_pm%i' % n
        cos_key = 'cos_pm%i' % n
                
        plot_thruster(ax=ax, x=position[0], y=position[1], cos=row[cos_key], sin=row[sin_key])
        
    
    ax.set_xlim(np.min(coordinates[:,0]),np.max(coordinates[:,0]))
    ax.set_xlim(np.min(coordinates[:,1]),np.max(coordinates[:,1]))
    ax.set_aspect('equal', 'box')
    
    ## Second plot:
    ax = axes[1]
    trip_.plot(x='longitude', y='latitude', ax=ax, style='k--')
    ax.plot(row['longitude'], row['latitude'], 'o')
    
    ax.set_ylim(trip_['latitude'].min(), trip_['latitude'].max())
    ax.set_xlim(trip_['longitude'].min(), trip_['longitude'].max())
    

def animate(i=0):
    
    index = int(i)
    row = trip_.iloc[index]
    plot_thrust_allocation(row=row)
        

        
interactive_plot = interactive(animate, i = (0,len(trip_)-1,1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot
    
#plot_thrust_allocation(row=trip.iloc[100])
    